# DNN Model Notebook

```md
@authors: miguelrocha and Grupo 03
```

In [1]:
# Notebook Imports
import numpy as np

from models.dnn_model import NeuralNetwork, hyperparameter_optimization
from helpers.dataset import Dataset
from helpers.enums import ModelRunMode
from helpers.layers import DenseLayer
from helpers.activation import ReLUActivation, SigmoidActivation
from helpers.losses import BinaryCrossEntropy
from helpers.optimizer import Optimizer
from helpers.metrics import accuracy
from helpers.enums import ModelType
from helpers.model import save_dnn_model

In [2]:
# Model run mode
# Options: 
#   ModelRunMode.TRAIN.value            (Train the model)
#   ModelRunMode.CLASSIFY.value         (Classify data)
mode = ModelRunMode.TRAIN.value
# Prefix for saving the model files
model_prefix = "dnn_model_1"

In [3]:
# Parameters cell
if mode == ModelRunMode.TRAIN.value:
    # TRAIN mode: Set parameters for training
    input_csv = "../tarefa_1/clean_input_datasets/dataset1_enh_inputs.csv"              # CSV file with training inputs (ID, Text)
    output_csv = "../tarefa_1/clean_output_datasets/dataset1_enh_outputs.csv"           # CSV file with training outputs (ID, Label)
    test_size = 0.3                                                                     # Proportion of the dataset to use as test data
    validation_size = 0.2                                                               # Proportion of the dataset reserved as validation data (used during hyperparameter tuning)
    verbose = True                                                                      # Verbosity level enabler
    random_state=42                                                                     # Seed for reproducible dataset splitting
    max_vocab_size=None                                                                 # Maximum vocabulary size (None implies no limit)
    min_freq=1                                                                          # Minimum frequency required for a word to be included in the vocabulary
elif mode == ModelRunMode.CLASSIFY.value:
    # CLASSIFY mode: Set parameters for classification
    input_csv = "../tarefa_2/classify_input_datasets/dataset2_inputs.csv"               # CSV file with texts for prediction (ID, Text)
    output_csv = "../tarefa_2/classify_output_datasets/dataset2_outputs_dnn_model.csv"  # CSV file to store prediction result
else:
    print("The selected option is not valid. Options: \"train\" or \"classify\"!")
    SystemExit()

In [4]:
if mode == ModelRunMode.TRAIN.value:
    # Load datasets using TF-IDF vectorization
    X_train, y_train, X_test, y_test, vocab, idf = Dataset.prepare_train_test_tfidf(input_csv=input_csv, output_csv=output_csv, test_size=test_size, random_state=random_state, max_vocab_size=max_vocab_size, min_freq=min_freq)

    # Create Dataset objects for training and test data
    train_ds_full = Dataset(X=X_train, Y=y_train)
    test_ds = Dataset(X=X_test, Y=y_test)

    # Display dimensions of the training and test datasets
    print(f"Train set has {train_ds_full.nrows()} rows and {train_ds_full.ncols()} columns")
    print(f"Test set has {test_ds.nrows()} rows and {test_ds.ncols()} columns\n")

    # Split the full training dataset into training and validation subsets (80% training, 20% validation)
    n_train = train_ds_full.X.shape[0]
    indices = np.arange(n_train)
    np.random.shuffle(indices)
    split_idx = int((1 - validation_size) * n_train)
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]
    train_ds = Dataset(X=train_ds_full.X[train_idx], Y=train_ds_full.Y[train_idx])
    val_ds = Dataset(X=train_ds_full.X[val_idx], Y=train_ds_full.Y[val_idx])

    # Define hyperparameter grids for tuning
    epochs_list = [100, 200, 300]
    batch_size_list = [16, 32]
    learning_rate_list = [0.001, 0.01, 0.1]
    momentum_list = [0.5, 0.8, 0.9]
    hidden_layers_list = [
        [32],
        [64],
        [50, 25],
        [64, 32],
        [128, 64]
    ]
    dropout_list = [0.0, 0.05, 0.1, 0.3, 0.5]

    # Perform hyperparameter tuning using the training and validation sets
    print("Starting hyperparameter tuning...")
    best_params = hyperparameter_optimization(train_ds, val_ds, epochs_list, batch_size_list, learning_rate_list, momentum_list, hidden_layers_list, dropout_list, n_iter=100)
    print("\nBest hyperparameters:", best_params)

    # Retrain DNN model on the full training data using the best hyperparameters
    final_model = NeuralNetwork(epochs=best_params['epochs'], batch_size=best_params['batch_size'], optimizer=Optimizer(learning_rate=best_params['learning_rate'], momentum=best_params['momentum']), verbose=verbose, loss=BinaryCrossEntropy,  metric=accuracy)

    # Build model Layers
    n_features = train_ds.X.shape[1]
    for i, units in enumerate(best_params['n_hidden']):
        if i == 0:
            final_model.add(DenseLayer(units, (n_features,),dropout_rate=best_params['dropout_rate']))
        else:
            final_model.add(DenseLayer(units,dropout_rate=best_params['dropout_rate']))
        final_model.add(ReLUActivation())
        
    final_model.add(DenseLayer(1))
    final_model.add(SigmoidActivation())

    # Fit model
    final_model.fit(train_ds)
    final_model.predict(test_ds)
    out = final_model.predict(test_ds)

    # Evaluate the final model on the test set
    acc = final_model.score(test_ds, out)
    print("Test Accuracy:", acc)

    # Save the model, plus vocab & idf, so classification can replicate
    save_dnn_model(dnn=final_model, vocab=vocab, idf=idf, model_prefix=model_prefix)
    print(f"Model saved with prefix {model_prefix}")

if mode == ModelRunMode.CLASSIFY.value:
    # Classify new texts using the saved model
    Dataset.classify_texts(input_csv=input_csv, output_csv=output_csv, neural_net_class=NeuralNetwork, model_type=ModelType.DNN ,model_prefix=model_prefix)

Train set has 70 rows and 2124 columns
Test set has 30 rows and 2124 columns

Starting hyperparameter tuning...


Hyperparameter Search:   1%|          | 1/100 [00:00<00:21,  4.61it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.1, 'momentum': 0.8, 'n_hidden': [32], 'dropout_rate': 0.5}  |  Accuracy: 0.4286


Hyperparameter Search:   2%|▏         | 2/100 [00:01<01:33,  1.05it/s]

Parameters: {'epochs': 100, 'batch_size': 16, 'learning_rate': 0.1, 'momentum': 0.9, 'n_hidden': [128, 64], 'dropout_rate': 0.0}  |  Accuracy: 0.6429


...

Hyperparameter Search:  99%|█████████▉| 99/100 [01:40<00:01,  1.62s/it]

Parameters: {'epochs': 200, 'batch_size': 16, 'learning_rate': 0.01, 'momentum': 0.9, 'n_hidden': [64, 32], 'dropout_rate': 0.1}  |  Accuracy: 0.6429


Hyperparameter Search: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it]

Parameters: {'epochs': 300, 'batch_size': 32, 'learning_rate': 0.01, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.3}  |  Accuracy: 0.7857

Best Hyperparameters Found: {'epochs': 300, 'batch_size': 32, 'learning_rate': 0.1, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.0}
Best Accuracy: 0.7857

Best hyperparameters: {'epochs': 300, 'batch_size': 32, 'learning_rate': 0.1, 'momentum': 0.8, 'n_hidden': [64], 'dropout_rate': 0.0}
Epoch 1/300 - loss: 0.7048 - accuracy: 0.5000
Epoch 2/300 - loss: 0.6896 - accuracy: 0.5625
Epoch 3/300 - loss: 0.6401 - accuracy: 0.7188
Epoch 4/300 - loss: 0.6831 - accuracy: 0.5312
Epoch 5/300 - loss: 0.6606 - accuracy: 0.5938
Epoch 6/300 - loss: 0.6339 - accuracy: 0.7188
Epoch 7/300 - loss: 0.6073 - accuracy: 0.7812
Epoch 8/300 - loss: 0.6241 - accuracy: 0.7812
Epoch 9/300 - loss: 0.5718 - accuracy: 0.7812
Epoch 10/300 - loss: 0.5823 - accuracy: 0.8125
Epoch 11/300 - loss: 0.5330 - accuracy: 0.9062
Epoch 12/300 - loss: 0.5616 - accuracy: 0.7500
E

Epoch 77/300 - loss: 0.0118 - accuracy: 1.0000
Epoch 78/300 - loss: 0.0126 - accuracy: 1.0000
Epoch 79/300 - loss: 0.0121 - accuracy: 1.0000
Epoch 80/300 - loss: 0.0132 - accuracy: 1.0000
Epoch 81/300 - loss: 0.0112 - accuracy: 1.0000
Epoch 82/300 - loss: 0.0113 - accuracy: 1.0000
Epoch 83/300 - loss: 0.0125 - accuracy: 1.0000
Epoch 84/300 - loss: 0.0096 - accuracy: 1.0000
Epoch 85/300 - loss: 0.0104 - accuracy: 1.0000
Epoch 86/300 - loss: 0.0112 - accuracy: 1.0000
Epoch 87/300 - loss: 0.0106 - accuracy: 1.0000
Epoch 88/300 - loss: 0.0097 - accuracy: 1.0000
Epoch 89/300 - loss: 0.0101 - accuracy: 1.0000
Epoch 90/300 - loss: 0.0090 - accuracy: 1.0000
Epoch 91/300 - loss: 0.0093 - accuracy: 1.0000
Epoch 92/300 - loss: 0.0094 - accuracy: 1.0000
Epoch 93/300 - loss: 0.0078 - accuracy: 1.0000
Epoch 94/300 - loss: 0.0092 - accuracy: 1.0000
Epoch 95/300 - loss: 0.0083 - accuracy: 1.0000
Epoch 96/300 - loss: 0.0089 - accuracy: 1.0000
Epoch 97/300 - loss: 0.0086 - accuracy: 1.0000
Epoch 98/300 